In [1]:
import numpy as np
import pywt
import scipy

In [2]:
#Change viewing options
from IPython.core.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

In [3]:
#pywt.wavelist()
#Information : http://wavelets.pybytes.com/

In [4]:
data = np.array(range(10))

In [5]:
#Set the number of levels for DWT, as well as the wavelet
data_len = data.size
levels = 1
wave = 'db2'

#figure out max possible levels to loop over given size of data
max_level = pywt.dwt_max_level(data_len = data_len, filter_len = pywt.Wavelet(wave).dec_len) 
max_level

1

# The following constructs the filter matrix using symmetric padding for the wavelet

In [6]:
def middle_match(data_len,wavelet):
    
    #Grab the filter and its length
    w_fil = pywt.Wavelet(wavelet).dec_lo
    w_fil = np.array(w_fil)
    w_fil = np.flip(w_fil, axis=0)
    w_len = pywt.Wavelet(wavelet).dec_len
    
    #Build the first row of the matrix
    I_m = np.zeros(data_len)
    I_m[:w_len] = w_fil
    
    #Make the shift matrix
    row_number = int(np.ceil((data_len-w_len)/2))
    if data_len%2 == 0:
        for j in range(1, row_number+1):
            row = np.zeros(data_len)
            row[2*j:w_len + 2*j]= w_fil
            I_m = np.concatenate((I_m,row), axis=0)
    
        I_m = np.reshape(I_m, (row_number+1,data_len))
    else:
        for j in range(1, row_number):
            row = np.zeros(data_len)
            row[2*j:w_len + 2*j]= w_fil
            I_m = np.concatenate((I_m,row), axis=0)
    
        I_m = np.reshape(I_m, (row_number,data_len))        
        
        
    return I_m

In [7]:
def top_match(data_len, wavelet):
    
    #Grab the filter and its length
    w_fil = pywt.Wavelet(wavelet).dec_lo
    w_fil = np.array(w_fil)
    w_fil = np.flip(w_fil, axis=0)
    w_len = pywt.Wavelet(wavelet).dec_len
    
    #Build blank matrix
    row_number = int(np.ceil((w_len)/2))
    I_t = np.zeros((row_number-1,data_len))
    
    #Make the shift matrix
    for j in range(1,row_number):
        overflow = w_fil[:2*j]
        overflow = np.flip(overflow, axis=0)
        row_o = np.zeros(data_len)
        row_o[:2*j] = overflow
        
        leftover = w_fil[2*j:]
        row_l = np.zeros(data_len)
        row_l[:leftover.size] = leftover
        
        row = row_o + row_l      
        
        I_t[j-1] = row
    
    return I_t

In [8]:
def bot_match(data_len, wavelet):
   
    #Grab the filter and its length
    w_fil = pywt.Wavelet(wavelet).dec_lo
    w_fil = np.array(w_fil)
    w_fil = np.flip(w_fil, axis=0)
    w_len = pywt.Wavelet(wavelet).dec_len

    row_number = int(np.ceil((w_len)/2))
    if data_len%2 ==0:
        I_b = np.zeros((row_number-1,data_len))
        for j in range(1,row_number):
            overflow = w_fil[-2*j:]
            overflow = np.flip(overflow, axis=0)
            row_o = np.zeros(data_len)
            row_o[-2*j:] = overflow
            
            
            leftover = w_fil[:-2*j]
            row_l = np.zeros(data_len)
            row_l[-leftover.size:] = leftover
            
            row = row_o + row_l
            
            I_b[j-1] = row         
    else:
        I_b = np.zeros((row_number,data_len))
        for j in range(1,row_number+1):         
            overflow = w_fil[-2*j+1:]
            overflow = np.flip(overflow, axis=0)
            row_o = np.zeros(data_len)
            row_o[-2*j+1:] = overflow
            
            leftover = w_fil[:-2*j+1]
            row_l = np.zeros(data_len)
            row_l[-leftover.size:] = leftover

            
            row = row_o + row_l
            
            I_b[j-1] = row
            
    return I_b

In [9]:
def filter_matrix(data_len, wavelet):
    
    I_m = middle_match(data_len, wavelet)
    I_t = top_match(data_len, wavelet)
    I_b = bot_match(data_len, wavelet)
    
    I = np.concatenate((I_t,I_m), axis=0)
    I = np.concatenate((I,I_b), axis=0)
    
    return I

In [10]:
I = filter_matrix(20,'haar')

In [11]:
I

array([[0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707]])

In [12]:
I.shape

(10, 20)

# The following discovers which class each point should belong to

In [13]:
I2 = np.abs(I)
I2

array([[0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.707, 0.707]])

In [14]:
classes_index = np.argmax(I2,axis=0)

In [15]:
classes_index

array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9])

In [16]:
I2[:,6]

array([0, 0, 0, 0.707, 0, 0, 0, 0, 0, 0])

In [33]:
#Function used to discover classes at one stage higher resolution
def map_back(I):
    I = np.abs(I)
    classes_index = np.argmax(I,axis=0)
    
    return classes_index

# Map back multiple levels

In [18]:
data = np.array(range(100))

In [19]:
#Set the number of levels for DWT, as well as the wavelet
data_len = data.size
levels = 2
wave = 'db2'

#figure out max possible levels to loop over given size of data
max_level = pywt.dwt_max_level(data_len = data_len, filter_len = pywt.Wavelet(wave).dec_len) 
max_level

5

In [22]:
w = pywt.wavedec(data,level = levels, wavelet=wave)

In [23]:
w

[array([0.692, 0.908, 7.8, 15.8, 23.8, 31.8, 39.8, 47.8, 55.8, 63.8, 71.8, 79.8, 87.8, 95.8, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 198]),
 array([-0.333, -0.296, -5.55e-17, -1.78e-15, 1.33e-15, 3.11e-15, 0, 0, 8.88e-16, 3.55e-15, 6.22e-15, 7.11e-15, -7.11e-15, -1.78e-15, -8.88e-15, -3.55e-15, -1.07e-14, 7.11e-15, -5.33e-15, 5.33e-15, 1.24e-14, 0, 5.33e-15, -3.55e-15, 1.42e-14, 1.42, 0.308]),
 array([-0.612, -2.78e-17, 1.11e-16, 4.44e-16, 0, 1.33e-15, 8.88e-16, 4.44e-16, 8.88e-16, 8.88e-16, 1.78e-15, 1.78e-15, 2.66e-15, 8.88e-16, 8.88e-16, 1.78e-15, 0, 2.66e-15, 8.88e-16, 0, 6.22e-15, 0, 2.66e-15, 1.78e-15, 2.66e-15, -1.78e-15, 4.44e-15, 0, 8.88e-16, 0, 2.66e-15, -3.55e-15, 1.78e-15, 5.33e-15, 5.33e-15, 1.78e-15, 0, 7.11e-15, 0, 5.33e-15, 7.11e-15, -7.11e-15, 0, 5.33e-15, 3.55e-15, -3.55e-15, 3.55e-15, -1.78e-15, 3.55e-15, 1.07e-14, 0.612])]

In [39]:
w[2].shape

(51,)

In [25]:
w[0].shape

(27,)

In [44]:
pywt.dwt_coeff_len(data.size, pywt.Wavelet(wave).dec_len, mode='symmetric')

51

In [42]:
pywt.dwt_coeff_len(51, pywt.Wavelet(wave).dec_len, mode='symmetric')

27

In [45]:
def dwt_length(data, wavelet):
    
    dwt_lengths = [data.size]
    max_level = pywt.dwt_max_level(data_len = data.size, filter_len = pywt.Wavelet(wave).dec_len)
    
    for i in range(max_level):
        l = pywt.dwt_coeff_len(dwt_lengths[i], pywt.Wavelet(wave).dec_len, mode='symmetric')
        dwt_lengths.append(l)
    
    return dwt_lengths

In [48]:
dwt_lengths = dwt_length(data, wave)
dwt_lengths

[100, 51, 27, 15, 9, 6]

In [64]:
I = filter_matrix(dwt_lengths[2],'db2')
I

array([[1.06, 0.354, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.483, 0.837, 0.224, -0.129, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [65]:
classes_index = map_back(I)
classes_index

array([ 0,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14])

In [66]:
classes_index.shape

(27,)

In [63]:
ex = np.array([0,0,0,0,4,5,2,2,2,1,1,1,1,1,3])
ex.shape

(15,)

In [69]:
ex2 = ex[classes_index]
ex2

array([0, 0, 0, 0, 0, 0, 4, 4, 5, 5, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])